In [1]:
import sys
import csv
import web_scrapping

In [2]:
###################
# opening csv file and put it's content into a list of urls
###################
csv_contents = []

with open("resources/furniture stores pages.csv", "r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        csv_contents.append(row[0])

csv_contents.remove("max(page)")

print(len(csv_contents))

704


In [4]:
###################
# Fetch HTML content for each URL
###################

html_contents = []

for url in csv_contents:
    html_content = web_scrapping.fetch_html_content(url)
    if html_content:
        html_contents.append(html_content)

print(f"Fetched HTML content from {len(html_contents)} URLs")

Error fetching content from https://furniturefetish.com.au/products/oslo-office-chair-white: HTTPSConnectionPool(host='furniturefetish.com.au', port=443): Max retries exceeded with url: /products/oslo-office-chair-white (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002324C9FFC40>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))
Failed to fetch content from https://hemisphereliving.com.au/products/. Status code:403
Failed to fetch content from https://home-buy.com.au/products/bridger-pendant-larger-lamp-metal-brass. Status code:404
Error fetching content from https://beckurbanfurniture.com.au/products/page/2/: HTTPSConnectionPool(host='beckurbanfurniture.com.au', port=443): Max retries exceeded with url: /products/page/2/ (Caused by SSLError(SSLCertVerificationE

In [5]:
###################
# Save fatched HTML content to file for faster following processing
###################

with open("resources/html_content.csv", "w", newline="") as f:
    write = csv.writer(f)
    write.writerow(html_contents)

In [2]:
###################
# setting new limit for csv reading because the html content si bigger that the actual size can read
###################

max_int = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(max_int)
        break
    except OverflowError:
        max_int = int(max_int/10)

In [16]:
###################
# open fatched HTML content to file for faster following processing
###################

with open("resources/html_content.csv", "r") as f:
    read = csv.reader(f)

    html_contents = list(read)[0]

In [4]:
print(html_contents[0])

b'<!doctype html>\n<html class="no-js" lang="en">\n  <head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1">\n    <meta name="theme-color" content="">\n    <meta name="google-site-verification" content="S4-p-M4ar3SxPdQrcXVG_IF3iM3C8cRx_FTaPLF4xjk" />\n    \n      <link rel="canonical" href="https://www.factorybuys.com.au/products/euro-top-mattress-king" />\n    \n    \n    <link rel="preconnect" href="https://cdn.shopify.com" crossorigin><link rel="icon" type="image/png" href="//www.factorybuys.com.au/cdn/shop/files/FB-Favicon_1.png?crop=center&height=32&v=1659852040&width=32"><link rel="preconnect" href="https://fonts.shopifycdn.com" crossorigin><title>\n      Factory Buys 32cm Euro Top Mattress - King\n</title>\n\n    \n      <meta name="description" content="Pamper yourself with Factory Buys Euro Top Mattress that will give you a soothing sleep night after n

In [46]:
###################
# preprocessing HTML content by removing unecessary tags
###################
import preprocessing_html

for i in range(len(html_contents)):
    html_contents[i] = preprocessing_html.preprocess_html(html_contents[i])
    # print(f"Preprocessed HTML content for analysis {i}/{len(html_contents)}")

# print(html_contents[0])
modified = html_contents[0].replace(r"\n", "\n")
modified = modified.replace("  ", "")
modified = modified.split("\n")
modified = [sentence for sentence in modified if sentence != "" and sentence != " "]
print(modified)
# print(max(modified, key=len))


["b'", 'Skip to content', 'FREE SHIPPINGALL MATTRESSES*', 'Buy Now Pay Later Available!', 'Fast Shipping Australia Wide!', 'Home Furniture', 'Home Furniture', 'Bedroom', 'Bedroom', 'Mattresses', 'Bed Frames', 'Bedroom Packages', 'Bedsides', 'Wardrobes', 'Tallboys', 'Dressing Tables', 'Headboards', 'Ottomans', 'Jewellery Cabinets', 'Dining', 'Dining', 'Bar Stools', 'Dining Chairs', 'Dining Tables', 'Dining Sets', 'Sideboards & Buffets', 'Kitchen Benches', 'Living Room', 'Living Room', 'Sofas', 'TV Units', 'Recliner Chairs', 'Coffee & Side Tables', 'Chairs', 'Massage Chairs', 'Hall Tables', 'Bean Bags', 'Arm Chairs', 'Futons & Daybeds', 'Living Room Packages', 'Office', 'Office', 'Office Chairs', 'Office Massage Chairs', 'Desks', 'Sit Stand Desks', 'Monitor Stands', 'Laptop Desks', 'Filing Cabinets', 'Office Accessories', 'Office Packages', 'LED Furniture', 'LED Furniture', 'LED Bed Frames ', 'LED TV Units ', 'LED Bedside Tables ', 'LED Desks ', 'LED Bedroom Packages', 'Gaming Furniture'